## モジュールのインポート

In [24]:
import pandas as pd
import numpy as np
from pathlib import Path
import re

## 変数定義

In [25]:
kaisai_date = "20241109"
year = kaisai_date[:4]

DATA_DIR = Path(".", "data", year, kaisai_date)
DATA_DIR.mkdir(exist_ok=True, parents=True)

KAKO_DATA_DIR = Path("C:/", "TFJV", "TXT", "data", "kako_data", year)
BASE_DATA_DIR = Path("C:/", "TFJV", "TXT", "data", "base_data", year)
PEDS_DATA_DIR = Path("C:/", "TFJV", "TXT", "data", "peds_data", year)

## ファイルの読み込み

In [26]:
zensou_df = pd.read_csv(KAKO_DATA_DIR / f"kako_data_{kaisai_date}.csv", encoding="shift-jis", header=None)
base_df = pd.read_csv(BASE_DATA_DIR / f"base_data_{kaisai_date}.csv", encoding="shift-jis", header=None)
peds_df = pd.read_csv(PEDS_DATA_DIR / f"peds_data_{kaisai_date}.csv", encoding="shift-jis", header=None)

In [27]:
zensou_df_pre = zensou_df.iloc[:, [8, 21, 25, 30, 43, 64]]
base_df_pre = base_df.iloc[:, [3, 4, 5, 7, 8, 9, 14, 21, 24, 26, 33]]
peds_df_pre = peds_df.iloc[:, [21, 32]]

# 前日などにCSVをダウンロードした時はこちらを使う
# base_df_pre = base_df.iloc[:, [3, 4, 5, 7, 8, 9, 14, 21, 24, 26, 31]]
# peds_df_pre = peds_df.iloc[:, [21, 30]]

## 前処理

### 前走データ

In [28]:
# カラム名を付与
zensou_df_pre.columns = ["距離", "馬名", "前走間隔", "前走距離", "前-2走前間隔", "2-3走前間隔"]

In [29]:
# 文字列型データを数値型データに変換
zensou_df_pre = zensou_df_pre.replace("連", 1)
zensou_df_pre = zensou_df_pre.replace("初", 0)

zensou_df_pre["前走間隔"] = pd.to_numeric(zensou_df_pre["前走間隔"], errors="coerce").astype(float)
zensou_df_pre["前-2走前間隔"] = pd.to_numeric(zensou_df_pre["前-2走前間隔"], errors="coerce").astype(float)
zensou_df_pre["2-3走前間隔"] = pd.to_numeric(zensou_df_pre["2-3走前間隔"], errors="coerce").astype(float)

In [30]:
int_cols = zensou_df_pre.select_dtypes(include=['int']).columns
zensou_df_pre[int_cols] = zensou_df_pre[int_cols].astype('float')

In [31]:
# 「中○週」の数値に変換
zensou_df_pre["前走間隔"] = zensou_df_pre["前走間隔"] - 1
zensou_df_pre["前-2走前間隔"] = zensou_df_pre["前-2走前間隔"] - 1
zensou_df_pre["2-3走前間隔"] = zensou_df_pre["2-3走前間隔"] - 1

In [32]:
# 「臨戦過程」の列を追加
import numpy as np

l = []
for i in range(len(zensou_df_pre)):
    if zensou_df_pre.loc[i, "前走間隔"] == 0:
        l.append("連闘")
    elif zensou_df_pre.loc[i, "前走間隔"] >= 12:
        l.append("休明初戦")
    elif zensou_df_pre.loc[i, "前-2走前間隔"] >= 12:
        l.append("休明2走")
    elif zensou_df_pre.loc[i, "2-3走前間隔"] >= 12:
        l.append("休明3走")
    else:
        l.append(np.nan)

zensou_df_pre["臨戦過程"] = l

In [33]:
# 「距離変遷」の列を追加
n = []
for i in range(len(zensou_df_pre)):
    if zensou_df_pre.loc[i, "前走距離"] - zensou_df_pre.loc[i, "距離"] > 0:
        n.append("距離短縮")
    elif zensou_df_pre.loc[i, "前走距離"] - zensou_df_pre.loc[i, "距離"] < 0:
        n.append("距離延長")
    else:
        n.append(np.nan)

zensou_df_pre["距離変遷"] = n

In [34]:
# 使用するカラムを絞る
zensou_df_pre = zensou_df_pre[[
    "馬名",
    "臨戦過程",
    "距離変遷"
]]

### 基本データ

In [35]:
# カラム名を付与
base_df_pre.columns = [
    "場所",
    "R",
    "クラス",
    "種別",
    "距離",
    "馬場状態",
    "馬番",
    "馬名",
    "年齢",
    "騎手名",
    "調教師名"
]

In [36]:
# 「クラス」列のデータを置換
base_df_pre['クラス'] = base_df_pre['クラス'].str.replace(r".*未勝利.*", "未勝利", regex=True)
base_df_pre['クラス'] = base_df_pre['クラス'].str.replace(r".*新馬.*", "新馬", regex=True)
base_df_pre['クラス'] = base_df_pre['クラス'].str.replace(r".*(１勝|２勝|３勝|1勝|2勝|3勝).*", "自己条件", regex=True)
base_df_pre['クラス'] = base_df_pre['クラス'].str.replace(r".*(G1|G2|G3).*", "重賞", regex=True)

condition = ~base_df_pre['クラス'].str.contains('新馬|未勝利|自己条件|重賞')
# base_df_pre.loc[condition, 'クラス'] = base_df_pre['クラス'].loc[condition].str.replace(r".*", "OP", regex=True)
base_df_pre.loc[condition, 'クラス'] = "OP"

In [37]:
# 距離区分の列を追加
l = []
for i in range(len(base_df_pre)):
    if base_df_pre.loc[i, "距離"] <= 1600:
        l.append("短距離")
    elif base_df_pre.loc[i, "距離"] <= 2200:
        l.append("中距離")
    else:
        l.append("長距離")

base_df_pre["距離区分"] = l

In [38]:
# 回りの区分の列を追加
l = []
left_tern = re.compile('東京|新潟|中京')

for i in range(len(base_df_pre)):
    if left_tern.search(base_df_pre.loc[i, "場所"]):
        l.append("左回り")
    else:
        l.append("右回り")

base_df_pre["回り"] = l

In [39]:
# 年齢データを変換
for i in range(2, 14):
    if i <= 4:
        base_df_pre['年齢'] = base_df_pre['年齢'].replace(i, f"{i}歳")
    else:
        base_df_pre['年齢'] = base_df_pre['年齢'].replace(i, "5歳以上")

In [40]:
base_df_pre["年齢"].value_counts()

年齢
2歳      226
3歳       96
4歳       85
5歳以上     81
Name: count, dtype: int64

In [41]:
# 馬場状態（道悪かどうか）の判定列を作る
l = []
ground_pattern = re.compile(r'.*良.*')

for i in range(len(base_df_pre)):
    if ground_pattern.search(base_df_pre.loc[i, "馬場状態"]):
        l.append(np.nan)
    else:
        if base_df_pre.loc[i, "種別"] == "芝":
            l.append("芝道悪")
        elif base_df_pre.loc[i, "種別"] == "ダート":
            l.append("ダ道悪")
        else:
            l.append(np.nan)

base_df_pre["道悪判定"] = l

### 血統データ

In [42]:
# カラム名を付与
peds_df_pre.columns = ["馬名", "種牡馬名"]
peds_df_pre

,馬名,種牡馬名
0,サードウインド,Dandy Man
1,ライヴドライバー,フォーウィールドライブ
2,ユウオウマイシン,ニシケンモノノフ
3,ゼンダンタカ,アジアエクスプレス
4,エソテリック,カレンブラックヒル
...,...,...
483,パープルクラウド,ドゥラメンテ
484,フォーチュンコード,スピルバーグ
485,キングサーガ,キングカメハメハ
486,リッチブラック,エピファネイア


## データの結合

In [43]:
base_df_preprocessed = base_df_pre.merge(peds_df_pre, on="馬名")

In [44]:
base_df_preprocessed = base_df_preprocessed.merge(zensou_df_pre, on="馬名")

In [45]:
base_df_preprocessed

,場所,R,クラス,種別,距離,馬場状態,馬番,馬名,年齢,騎手名,調教師名,距離区分,回り,道悪判定,種牡馬名,臨戦過程,距離変遷
0,福島,1,未勝利,ダート,1150,良,1,サードウインド,2歳,丹内祐次,西田雄一,短距離,右回り,NaN,Dandy Man,NaN,距離短縮
1,福島,1,未勝利,ダート,1150,良,2,ライヴドライバー,2歳,吉田隼人,尾形和幸,短距離,右回り,NaN,フォーウィールドライブ,NaN,距離短縮
2,福島,1,未勝利,ダート,1150,良,3,ユウオウマイシン,2歳,長浜鴻緒,松永康利,短距離,右回り,NaN,ニシケンモノノフ,NaN,距離短縮
3,福島,1,未勝利,ダート,1150,良,4,ゼンダンタカ,2歳,角田大和,角田晃一,短距離,右回り,NaN,アジアエクスプレス,NaN,距離短縮
4,福島,1,未勝利,ダート,1150,良,5,エソテリック,2歳,嶋田純次,矢嶋大樹,短距離,右回り,NaN,カレンブラックヒル,NaN,距離短縮
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
483,京都,12,自己条件,芝,2400,良,2,パープルクラウド,4歳,鮫島克駿,吉村圭司,長距離,右回り,NaN,ドゥラメンテ,休明初戦,距離短縮
484,京都,12,自己条件,芝,2400,良,3,フォーチュンコード,4歳,岩田望来,小林真也,長距離,右回り,NaN,スピルバーグ,休明3走,距離延長
485,京都,12,自己条件,芝,2400,良,4,キングサーガ,5歳以上,Ｃ．デム,小栗実,長距離,右回り,NaN,キングカメハメハ,休明2走,距離延長
486,京都,12,自己条件,芝,2400,良,5,リッチブラック,5歳以上,松山弘平,西村真幸,長距離,右回り,NaN,エピファネイア,休明3走,NaN


## ファイル出力

In [46]:
base_df_preprocessed.to_csv(DATA_DIR / f"preprocessed_data_{kaisai_date}.csv", index=False)